In [1]:
import pandas as pd 

In [2]:
df=pd.read_csv("/kaggle/input/fakenews/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df=df.dropna()

In [5]:
df.shape

(18285, 5)

In [6]:
X=df.drop('label', axis=1)  ##independent features

In [7]:
y=df['label']  ##dependent feature

In [8]:
X.shape

(18285, 4)

In [9]:
import tensorflow as tf

2024-02-02 11:14:26.806353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-02 11:14:26.806472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-02 11:14:26.993383: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense, LSTM, Bidirectional


In [11]:
v_size=5000  ##vocabulary size

In [12]:
messages=X.copy()


In [13]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [14]:
messages.reset_index(inplace=True)

In [15]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
messages.title[0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [19]:
###Preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]

for i in range(0, len(messages)):
    review= re.sub('[^a-zA-Z]',' ',messages.title[i])  ##substitutes string that are not in a-z and A-Z range with a blankspace
    review=review.lower()
    review= review.split()
    
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')] ##list: to stem words(that aren't stopwords) to their base form
    review=' '.join(review)
    corpus.append(review)  
    

In [20]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [21]:
###One-hot Representation
onehot=[one_hot(word,v_size) for word in corpus]
onehot

[[82, 2254, 4984, 2761, 304, 2641, 1922, 3888, 934, 1896],
 [840, 161, 1990, 1790, 666, 1576, 3923],
 [1265, 1908, 1272, 2766],
 [2947, 2353, 1518, 4847, 3183, 3397],
 [3017, 666, 1479, 1811, 416, 2684, 666, 3471, 1268, 3942],
 [34,
  2167,
  2595,
  3427,
  1483,
  2361,
  3941,
  3694,
  1143,
  806,
  1230,
  1159,
  1963,
  3213,
  3923],
 [1923, 2149, 3285, 1018, 3914, 2816, 4791, 1280, 2590, 3677, 4371],
 [1366, 4823, 3761, 3813, 63, 826, 2361, 4051, 2590, 3677, 4371],
 [4270, 1422, 1788, 2218, 1152, 2186, 635, 4470, 2361, 4777],
 [2584, 2047, 2042, 3471, 704, 4946, 1460, 2267],
 [2858, 4551, 2277, 4016, 1054, 4511, 2334, 4397, 2818, 4214, 3049],
 [4847, 3079, 304, 2186, 2361, 63],
 [652, 1483, 2144, 4075, 2614, 2833, 4541, 2148, 1634],
 [2736, 3585, 4466, 1058, 1199, 204, 3611, 2590, 3677, 4371],
 [3853, 2894, 4112, 1029, 3302, 2590, 3677, 4371],
 [1050, 2833, 3561, 2646, 261, 4308, 3387, 3315, 1188, 2989],
 [3566, 2510, 161],
 [4738, 1031, 2315, 4914, 2361, 1878, 4026, 3923],
 

In [22]:
##Embedding Representation
slen=20
emb=pad_sequences(onehot, padding='pre', maxlen=slen)
print(emb)

[[   0    0    0 ... 3888  934 1896]
 [   0    0    0 ...  666 1576 3923]
 [   0    0    0 ... 1908 1272 2766]
 ...
 [   0    0    0 ... 2590 3677 4371]
 [   0    0    0 ...  533 1184 2986]
 [   0    0    0 ... 4597 1673 3143]]


In [23]:
len(emb)
emb[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   82,
       2254, 4984, 2761,  304, 2641, 1922, 3888,  934, 1896], dtype=int32)

In [24]:
### Model Building
emb_vector_features=40   ##each word can be rep as a dense vector
model=Sequential()
model.add(Embedding(v_size, emb_vector_features, input_length= slen))
model.add(LSTM(200))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 200)               192800    
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 393001 (1.50 MB)
Trainable params: 393001 (1.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [25]:
import numpy as np
X_final= np.array(emb)
y_final= np.array(y)


In [26]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [29]:
### Training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 12s 50ms/step - loss: 0.3181 - accuracy: 0.8544 - val_loss: 0.1933 - val_accuracy: 0.9205
Epoch 2/10
192/192 [==============================] - 9s 46ms/step - loss: 0.1368 - accuracy: 0.9465 - val_loss: 0.1965 - val_accuracy: 0.9234
Epoch 3/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0918 - accuracy: 0.9655 - val_loss: 0.2282 - val_accuracy: 0.9181
Epoch 4/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0615 - accuracy: 0.9777 - val_loss: 0.2849 - val_accuracy: 0.9155
Epoch 5/10
192/192 [==============================] - 9s 47ms/step - loss: 0.0423 - accuracy: 0.9862 - val_loss: 0.2808 - val_accuracy: 0.9044
Epoch 6/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0277 - accuracy: 0.9909 - val_loss: 0.3733 - val_accuracy: 0.9176
Epoch 7/10
192/192 [==============================] - 9s 45ms/step - loss: 0.0160 - accuracy: 0.9947 - val_loss: 0.4364 - val_accuracy: 0.916

In [30]:
### Adding DROPOUT
from tensorflow.keras.layers import Dropout

emb_vector_features=40
model=Sequential()
model.add(Embedding(v_size, emb_vector_features, input_length= slen))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout (Dropout)           (None, 20, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 200)               192800    
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 393001 (1.50 MB)
Trainable params: 393001 (1.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [31]:
### Training
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
y_pred=model.predict(X_test)

Epoch 1/10
192/192 [==============================] - 12s 51ms/step - loss: 0.3390 - accuracy: 0.8378 - val_loss: 0.1988 - val_accuracy: 0.9152
Epoch 2/10
192/192 [==============================] - 10s 52ms/step - loss: 0.1511 - accuracy: 0.9413 - val_loss: 0.1882 - val_accuracy: 0.9261
Epoch 3/10
192/192 [==============================] - 9s 48ms/step - loss: 0.1089 - accuracy: 0.9593 - val_loss: 0.2042 - val_accuracy: 0.9251
Epoch 4/10
192/192 [==============================] - 10s 53ms/step - loss: 0.0811 - accuracy: 0.9710 - val_loss: 0.2263 - val_accuracy: 0.9208
Epoch 5/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0609 - accuracy: 0.9781 - val_loss: 0.2473 - val_accuracy: 0.9203
Epoch 6/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0437 - accuracy: 0.9854 - val_loss: 0.3473 - val_accuracy: 0.9176
Epoch 7/10
192/192 [==============================] - 9s 48ms/step - loss: 0.0325 - accuracy: 0.9891 - val_loss: 0.3409 - val_accuracy: 0.9

In [32]:
y_pred=(y_pred>0.5)

In [33]:
y_pred=y_pred.astype(int)

In [34]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm=confusion_matrix(y_test, y_pred)
cm

array([[3157,  262],
       [ 271, 2345]])

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9116818558409279

In [36]:
### Bidirectional LSTM

emb_vector_features=40
model1=Sequential()
model1.add(Embedding(v_size, emb_vector_features, input_length= slen))
model1.add(Dropout(0.3))
model1.add(Bidirectional(LSTM(200)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 bidirectional (Bidirection  (None, 400)               385600    
 al)                                                             
                                                                 
 dropout_3 (Dropout)         (None, 400)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 401       
                                                                 
Total params: 586001 (2.24 MB)
Trainable params: 586001 (2.24 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [37]:
model1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)
y_pred1=model1.predict(X_test)

Epoch 1/10
192/192 [==============================] - 18s 73ms/step - loss: 0.2976 - accuracy: 0.8595 - val_loss: 0.2045 - val_accuracy: 0.9153
Epoch 2/10
192/192 [==============================] - 13s 68ms/step - loss: 0.1500 - accuracy: 0.9404 - val_loss: 0.2035 - val_accuracy: 0.9213
Epoch 3/10
192/192 [==============================] - 13s 70ms/step - loss: 0.1086 - accuracy: 0.9584 - val_loss: 0.2230 - val_accuracy: 0.9153
Epoch 4/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0860 - accuracy: 0.9681 - val_loss: 0.2525 - val_accuracy: 0.9147
Epoch 5/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0623 - accuracy: 0.9771 - val_loss: 0.3043 - val_accuracy: 0.9158
Epoch 6/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0469 - accuracy: 0.9832 - val_loss: 0.3318 - val_accuracy: 0.9095
Epoch 7/10
192/192 [==============================] - 14s 74ms/step - loss: 0.0385 - accuracy: 0.9861 - val_loss: 0.3204 - val_accuracy:

In [38]:
y_pred1=(y_pred1>0.5)
y_pred1=y_pred1.astype(int)

In [39]:
from sklearn.metrics import confusion_matrix, classification_report
cm=confusion_matrix(y_test, y_pred1)
cm
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred1)

0.9091963545981773